In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
# Load the saved model
model = load_model('my_model.h5')


2024-04-25 18:08:47.303706: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 18:08:47.308710: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 18:08:47.585450: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 18:08:52.688560: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-25 18:08:59.921728: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/in

In [2]:
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load new data
new_data_df = pd.read_csv('clean_test_or_not_test_debt.csv')
new_data_df.head()


,Unnamed: 0,text_clean,label
0,0,make sure integration tests are running with i...,1
1,1,cover anvil service response in unit testshttp...,1
2,2,service apprepositorycontroller should be cach...,1
3,3,many caches persist between test casesan examp...,1
4,4,improve friendliness of behat text pattern mat...,1


In [3]:
new_texts = new_data_df['text_clean'].values  # Adjust column name as necessary
tokenizer = Tokenizer(num_words=5000)  # Adjust `num_words` as needed
tokenizer.fit_on_texts(new_texts)
# Tokenize and pad sequences

sequences = tokenizer.texts_to_sequences(new_texts)
max_sequence_length = max(len(x) for x in sequences)
X_new = pad_sequences(sequences, maxlen=max_sequence_length)

In [4]:
predictions = model.predict(X_new)
predicted_classes = (predictions > 0.5).astype(int).flatten()  # Adjust threshold as needed


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step


In [5]:
new_data_df['predicted_label'] = predicted_classes
new_data_df.to_csv('classified_external_data.csv', index=False)


In [6]:
import pandas as pd

classified_data_df = pd.read_csv('classified_external_data.csv')
y_true = classified_data_df['label'].values
y_pred = classified_data_df['predicted_label'].values


In [7]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='binary')  # Use 'macro' or 'weighted' for multi-class classification
precision = precision_score(y_true, y_pred, average='binary')  # Adjust as necessary
recall = recall_score(y_true, y_pred, average='binary')  # Adjust as necessary

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')


Accuracy: 0.4531
F1 Score: 0.4097
Precision: 0.4471
Recall: 0.3780
